In [19]:
import argparse

import torch
import torchvision

from dataset import GNHK, GNHKDataset
from model import Model
from utils import CTCLabelConverter, display_image, get_box_images

def collate_fn(batch):
    return tuple(zip(*batch))

In [20]:
import numpy as np
import gradio as gr

def inference(input_img):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    crnn_model = Model()
    print('loading pretrained model from %s' % 'text-recognition.pth')
    crnn_model.load_state_dict(torch.load('text-recognition.pth', map_location=device))

    # Converter
    converter = CTCLabelConverter()

    crnn_model.to(device)
    crnn_model.eval()
    
    img = torchvision.transforms.functional.to_tensor(input_img)
    transform = torchvision.transforms.Resize((32,32))
    img_list = [transform(img)]
    img = torch.stack(img_list).to(device)
    
    crnn_preds = crnn_model(img)
    preds_size = torch.IntTensor([crnn_preds.size(1)])

    crnn_preds = crnn_preds.log_softmax(2)

    _, preds_index = crnn_preds.max(2)
    preds_str = converter.decode(preds_index.data, preds_size.data)
    
    return preds_str[0]

demo = gr.Interface(inference, gr.Image(shape=(32, 32)), "text")
demo.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


loading pretrained model from text-recognition.pth
loading pretrained model from text-recognition.pth
loading pretrained model from text-recognition.pth
loading pretrained model from text-recognition.pth
loading pretrained model from text-recognition.pth
